In [24]:
import csv
import datetime
import operator, itertools, functools

In [3]:
guns = list(csv.reader(open('guns.csv', 'r')))
data = guns[1:]
headers = guns[0]

for row in data:
    row[1] = int(row[1])
    row[2] = int(row[2])

In [15]:
print(headers)
print(data[:5])

['', 'year', 'month', 'intent', 'police', 'sex', 'age', 'race', 'hispanic', 'place', 'education']
[['1', 2012, 1, 'Suicide', '0', 'M', '34', 'Asian/Pacific Islander', '100', 'Home', '4'], ['2', 2012, 1, 'Suicide', '0', 'F', '21', 'White', '100', 'Street', '3'], ['3', 2012, 1, 'Suicide', '0', 'M', '60', 'White', '100', 'Other specified', '4'], ['4', 2012, 2, 'Suicide', '0', 'M', '64', 'White', '100', 'Home', '4'], ['5', 2012, 2, 'Suicide', '0', 'M', '31', 'White', '100', 'Other specified', '2']]


In [21]:
def count_dict(func, iterable):
    counts = {}
    for item in iterable:
        key = func(item)
        counts[key] = counts[key]+1 if key in counts else 1
    return counts

In [22]:
year_counts = count_dict(lambda x: x[1], data)
print(year_counts)

{2012: 33563, 2013: 33636, 2014: 33599}


In [26]:
def date_key_func(row):
    dt = datetime.datetime(year=row[1], month=row[2], day=1)
    return dt.strftime('%Y-%m-%d')
date_counts = count_dict(date_key_func, data)
# print(date_counts)

In [30]:
sex_counts = count_dict(lambda x: x[5], data)
race_counts = count_dict(lambda x: x[7], data)
print(sex_counts)
print(race_counts)
print(sex_counts['M'] / sex_counts['F'])

{'F': 14449, 'M': 86349}
{'Black': 23296, 'White': 66237, 'Hispanic': 9022, 'Native American/Native Alaskan': 917, 'Asian/Pacific Islander': 1326}
5.976122915080628


# 分かったこと
* 銃で死亡する男性の数は女性の6倍
* 銃による死亡は白人が最も多く、ついで黒人

アメリカの人種の分布と比べて見たい

In [38]:
race_census = list(csv.reader(open('census.csv', 'r')))
print(race_census)

[['Id', 'Year', 'Id', 'Sex', 'Id', 'Hispanic Origin', 'Id', 'Id2', 'Geography', 'Total', 'Race Alone - White', 'Race Alone - Hispanic', 'Race Alone - Black or African American', 'Race Alone - American Indian and Alaska Native', 'Race Alone - Asian', 'Race Alone - Native Hawaiian and Other Pacific Islander', 'Two or More Races'], ['cen42010', 'April 1, 2010 Census', 'totsex', 'Both Sexes', 'tothisp', 'Total', '0100000US', '', 'United States', '308745538', '197318956', '44618105', '40250635', '3739506', '15159516', '674625', '6984195']]


In [43]:
race_census = list(csv.reader(open('census.csv', 'r')))[1][10:]
race_census = list(map(int, race_census))
mapping = {
    'Black': race_census[2], 'White': race_census[0],
    'Hispanic': race_census[1],
    'Native American/Native Alaskan': race_census[3],
    'Asian/Pacific Islander': race_census[4] + race_census[5]
}
print(mapping)

{'Black': 40250635, 'Hispanic': 44618105, 'White': 197318956, 'Asian/Pacific Islander': 15834141, 'Native American/Native Alaskan': 3739506}


In [46]:
def calc_rate_per_hundredk(item):
    race, killed = item
    return (race, killed / mapping[race] * 100000)
race_per_hundredk = dict(
    map(calc_rate_per_hundredk, race_counts.items()))
print(race_per_hundredk)

{'Black': 57.8773477735196, 'Hispanic': 20.220491210910907, 'White': 33.56849303419181, 'Asian/Pacific Islander': 8.374309664161762, 'Native American/Native Alaskan': 24.521955573811088}


In [49]:
data_only_homicide = filter(lambda x: x[3] == 'Homicide', data)
homicide_race_counts = count_dict(lambda x: x[7], data_only_homicide)
homicide_race_per_hundredk = dict(
    map(calc_rate_per_hundredk, homicide_race_counts.items()))
print(homicide_race_per_hundredk)

{'White': 4.6356417981453335, 'Hispanic': 12.627161104219914, 'Black': 48.471284987180944, 'Asian/Pacific Islander': 3.530346230970155, 'Native American/Native Alaskan': 8.717729026240365}


# 分かったこと
* 最も銃で殺されることが多い人種は黒人で、少ない人種はアジア・太平洋系
* 白人は二番目に銃で殺されにくい人種